In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
import os

In [3]:
from scipy.io import loadmat
output = loadmat(os.path.join('..','ssnmf','data','sonar_output.mat'))

In [ ]:
init_W = output['params_all']['init_W'][0,0]
init_H = output['params_all']['init_H'][0,0]
max_iter = output['params_all']['max_iter'][0,0]

In [4]:
output['LL'].shape

(15984, 62)

Testing parameters:

r = 3 

sm = 10000000, sp = 100

steps = 200

In [ ]:
%%time
# testing no sparsity, no smoothnes, 1 iteration
import ssnmf
model = ssnmf.smoothNMF(r=3, max_iter=1000, betaW=0, betaH=0, smoothness=10000000, sparsity=100)



# print(LA.norm(output['LL'] - (model.W@model.H)))

In [ ]:
from ssnmf.ssnmf import _update_H

In [ ]:
#!conda install --yes line_profiler
%load_ext line_profiler

In [ ]:
%lprun -f _update_H model.fit(output['LL'], W=init_W, H=init_H, init = 'custom')

In [ ]:
%%time 
model.fit(output['LL'], W=init_W, H=init_H, init='custom')

In [ ]:
plt.plot(model.cost)

In [ ]:
np.sqrt(np.trace((model.W.T @ model.W) @ (model.W.T @ model.W)))

In [ ]:
import scipy.linalg as LA
LA.norm(model.W @ model.W.T)

In [ ]:
# using tensorly to reshape
import tensorly as tl

In [ ]:
model.W.shape

In [ ]:

# Average across time of day columns
tod_num = 6

In [ ]:
W_reshaped = model.W.reshape(3, 144, 37, 3)

In [ ]:
LL_reconstructed = (model.W[:,:].reshape([-1,3])@model.H[:,:].reshape([3,-1])).reshape((3,144,37,62))

In [ ]:
plt.imshow(output["LL"][:,0].reshape(3,144,37)[0,:,:].T)


In [ ]:
depth_bin_size = 5
tod_num = 1

Sonar time series reconstructed with 3 components.

In [ ]:
plt.figure(figsize=(16,4.5))
freq = [38,120,200]

fig,ax = plt.subplots(3,1,figsize=(18,7),sharex=True)
for f in range(3):
    ax[f].imshow(tl.unfold(LL_.transpose(0,2,3,1)[f,:,:,:],0)[:,:][::-1,:],aspect='auto')
    ax[f].set_xticks(range(0,int(62*144/tod_num),int(144/tod_num*5)))
    ax[f].set_xticklabels(range(0,65,5))
    ax[f].set_yticks(range(0,37,5))
    ax[f].set_yticklabels((np.arange(0,37,5)+1)*depth_bin_size)
    plt.setp(ax[f].get_xticklabels(), fontsize=12)
    plt.setp(ax[f].get_yticklabels(), fontsize=12)
    if ax[f].is_last_row():
        plt.xlabel('Day',fontsize=14)
    ax[f].set_ylabel('Depth (m)',fontsize=14)
    ax[f].set_title('Frequency = %d kHz' % freq[f], fontsize=14)


Sonar time series after PCP.

In [ ]:
plt.figure(figsize=(16,4.5))
freq = [38,120,200]

fig,ax = plt.subplots(3,1,figsize=(18,7),sharex=True)
for f in range(3):
    ax[f].imshow(tl.unfold(output["LL"].reshape(3,144,37,62).transpose(0,2,3,1)[f,:,:,:],0)[:,:][::-1,:],aspect='auto')
    ax[f].set_xticks(range(0,int(62*144/tod_num),int(144/tod_num*5)))
    ax[f].set_xticklabels(range(0,65,5))
    ax[f].set_yticks(range(0,37,5))
    ax[f].set_yticklabels((np.arange(0,37,5)+1)*depth_bin_size)
    plt.setp(ax[f].get_xticklabels(), fontsize=12)
    plt.setp(ax[f].get_yticklabels(), fontsize=12)
    if ax[f].is_last_row():
        plt.xlabel('Day',fontsize=14)
    ax[f].set_ylabel('Depth (m)',fontsize=14)
    ax[f].set_title('Frequency = %d kHz' % freq[f], fontsize=14)


In [ ]:
#freq = [38,120,200]

#fig,ax = plt.subplots(3,1,figsize=(18,7),sharex=True)
#for f in range(3):
#    ax[f].imshow(tl.unfold(tl.tensor(out[:,:,:]),mode=1)[::-1,:],
#                           aspect='auto', vmin=-80, vmax=-40, cmap='jet')
#    ax[f].set_xticks(range(0,int(62*144/tod_num),int(144/tod_num*5)))
#    ax[f].set_xticklabels(range(0,65,5))
#    ax[f].set_yticks(range(0,37,5))
#    ax[f].set_yticklabels((np.arange(0,37,5)+1)*depth_bin_size)
#    plt.setp(ax[f].get_xticklabels(), fontsize=12)
#    plt.setp(ax[f].get_yticklabels(), fontsize=12)
#    if ax[f].is_last_row():
#        plt.xlabel('Day',fontsize=14)
#    ax[f].set_ylabel('Depth (m)',fontsize=14)
#    ax[f].set_title('Frequency = %d kHz' % freq[f], fontsize=14)

In [ ]:
# # a function for transforming the data for plotting
# def separate_transform_result(D,ori_data,ping_per_day_mvbs,log_opt=1):
#    '''
#    Separate transformed results into different frequencies and 
#    for use with `plot_cmp_data_decomp` and `plot_single_day`
#    '''
#    D_long = D.reshape((D.shape[0],-1,ori_data.shape[1])).swapaxes(1,2)
#    D_sep = D_long.reshape((D_long.shape[0],D_long.shape[1],-1,ping_per_day_mvbs)).transpose((2,0,1,3))
#    if log_opt==1:
#        D_plot = 10*np.log10(D_sep.transpose((0,2,1,3))).reshape((D_sep.shape[0],D_sep.shape[2],-1))
#    else:
#        D_plot = D_sep.transpose((0,2,1,3)).reshape((D_sep.shape[0],D_sep.shape[2],-1))
#    return D_sep,D_plot

In [ ]:
# L_sep, L_plot = separate_transform_result(output['LL'],mvbs,ping_per_day_mvbs,log_opt = 0)

In [ ]:
#plt.figure(figsize=(16,4.5))
#plt.imshow(np.moveaxis(output["LL"][:,:].reshape(3,144,37,62),1,2)[0,:,:,:].reshape(37,-1)[:,:500])

In [ ]:
#plt.figure(figsize=(16,4.5))
#plt.imshow(np.moveaxis(output["LL"][:,:].reshape(144,37,62),1,2)[0,:,:,:].reshape(37,-1)[:,:500])

In [ ]:
#plt.figure(figsize=(16,4.5))
#plt.imshow(np.moveaxis(B[:,:].reshape(3,144,37,62),1,2).reshape(3,37,-1)[2,:,:500])

In [ ]:
model.W.shape

In [ ]:
# coefficients in time
plt.plot(model.H.T)

In [ ]:
model = ssnmf.smoothNMF(r=3, max_iter=10, betaW=0, betaH=0, smoothness=0)

%prun model.fit(output['LL'], W=init_W, H=init_H, init='custom')

# print(LA.norm(output['LL'] - (model.W@model.H)))

# plt.plot(model.cost)

In [ ]:
from ssnmf.ssnmf import _update_H

In [ ]:
%lprun -f _update_H model.fit(output['LL'], W=init_W, H=init_H, init='custom')

In [ ]:
plt.plot(model.W)

In [ ]:
# Compare with WH outcomes from Matlab
WH = loadmat('/Users/valentina/Downloads/WH.mat')
print(np.max(np.abs(WH['H'] - model.H)))
print(np.max(np.abs(WH['W'] - model.W)))

In [ ]:
WH_cost = loadmat('/Users/valentina/Downloads/WH_cost.mat')
print(np.max(np.abs(WH_cost['H'] - model.H)))
print(np.max(np.abs(WH_cost['W'] - model.W)))
print(np.max(np.abs(WH_cost['objective'].ravel() - np.array(model.cost))))

In [ ]:
# Cost calculation error

In [ ]:
plt.plot(WH_cost['objective'].ravel()-np.array(model.cost),'ro')
#plt.plot(model.cost)
plt.title('Cost Calculation Error')

In [ ]:
WH_cost['objective'].shape
np.array(model.cost).shape

In [ ]:
WH_cost.keys()

### Rank Selection & Cophenetic Correlation

In [ ]:
import ssnmf
model = ssnmf.smoothNMF(r=3, max_iter=100, betaW=0, betaH=0, smoothness=1000000, sparsity=10)
model.fit(output['LL'], init='random')

In [ ]:
    # from Nimfa package

    def coph_cor(self, idx=None):
        """
        Compute cophenetic correlation coefficient of consensus matrix, generally obtained from multiple NMF runs. 
        
        The cophenetic correlation coefficient is measure which indicates the dispersion of the consensus matrix and is based 
        on the average of connectivity matrices. It measures the stability of the clusters obtained from NMF. 
        It is computed as the Pearson correlation of two distance matrices: the first is the distance between samples induced by the 
        consensus matrix; the second is the distance between samples induced by the linkage used in the reordering of the consensus 
        matrix [Brunet2004]_.
        
        Return real number. In a perfect consensus matrix, cophenetic correlation equals 1. When the entries in consensus matrix are
        scattered between 0 and 1, the cophenetic correlation is < 1. We observe how this coefficient changes as factorization rank 
        increases. We select the first rank, where the magnitude of the cophenetic correlation coefficient begins to fall [Brunet2004]_.
        
        :param idx: Used in the multiple NMF model. In factorizations following standard NMF model or nonsmooth NMF model
                    :param:`idx` is always None.
        :type idx: None or `str` with values 'coef' or 'coef1' (`int` value of 0 or 1, respectively) 
        """
        A = self.consensus(idx=idx)
        # upper diagonal elements of consensus
        avec = np.array([A[i, j] for i in range(A.shape[0] - 1)
                        for j in range(i + 1, A.shape[1])])
        # consensus entries are similarities, conversion to distances
        Y = 1 - avec
        Z = linkage(Y, method='average')
        # cophenetic correlation coefficient of a hierarchical clustering
        # defined by the linkage matrix Z and matrix Y from which Z was
        # generated
        return cophenet(Z, Y)[0]

In [ ]:
from operator import eq
def elop(X, Y, op):
    #try:
    #    X[X == 0] = np.finfo(X.dtype).eps
    #    Y[Y == 0] = np.finfo(Y.dtype).eps
    #except ValueError:
    #    return op(np.mat(X), np.mat(Y))
    return(op(np.mat(X), np.mat(Y)))

In [ ]:
    def connectivity(model, X, H=None, idx=None):
        """
        Compute the connectivity matrix for the samples based on their mixture coefficients. 
        
        The connectivity matrix C is a symmetric matrix which shows the shared membership of the samples: entry C_ij is 1 iff sample i and 
        sample j belong to the same cluster, 0 otherwise. Sample assignment is determined by its largest metagene expression value. 
        
        Return connectivity matrix.
        
        :param idx: Used in the multiple NMF model. In factorizations following
            standard NMF model or nonsmooth NMF model ``idx`` is always None.
        :type idx: None or `str` with values 'coef' or 'coef1' (`int` value of 0 or 1, respectively) 
        """
        #X = model.X
        H = model.H
        idx = np.argmax(H, axis=0)
        #plt.imshow(H)
        plt.plot(H.T)
        print(idx)
        mat1 = np.tile(idx, (X.shape[1], 1))
        mat2 = np.tile(np.reshape(idx.T,(len(idx),1)), (1, X.shape[1]))
        conn = elop(mat1, mat2, eq)
        #if sp.isspmatrix(conn):
        #    return conn.__class__(conn, dtype='d')
        #else:
        return np.mat(conn, dtype='d')

In [ ]:
res = connectivity(model, output['LL'])

In [ ]:
res.min()

In [ ]:
def consensus(n_run, X, model):
    """
    Compute consensus matrix as the mean connectivity matrix across multiple runs of the factorization. It has been
    proposed by [Brunet2004]_ to help visualize and measure the stability of the clusters obtained by NMF.
        
    Tracking of matrix factors across multiple runs must be enabled for computing consensus matrix. For results
    of a single NMF run, the consensus matrix reduces to the connectivity matrix.
        
    :param idx: Used in the multiple NMF model. In factorizations following
            standard NMF model or nonsmooth NMF model ``idx`` is always None.
    :type idx: None or `str` with values 'coef' or 'coef1' (`int` value of 0 or 1, respectively) 
    """
        #V = self.target(idx)
        #if self.track_factor:
        #    if sp.isspmatrix(V):
        #        cons = V.__class__((V.shape[1], V.shape[1]), dtype=V.dtype)
        #    else:
        #        cons = np.mat(np.zeros((V.shape[1], V.shape[1])))
        #    for i in range(self.n_run):
        #        cons += self.connectivity(
        #            H=self.tracker.get_factor(i).H, idx=idx)
        #    return sop(cons, self.n_run, div)
        #else:
        #    return self.connectivity(H=self.coef(idx), idx=idx)
    cons = np.mat(np.zeros((X.shape[1], X.shape[1])))
    for run in range(n_run):
        model.fit(X, init='random')
        cons += connectivity(model, X)
    return(cons/n_run)

In [ ]:
cons_matrix = consensus(5, output['LL'], model)

In [ ]:
plt.imshow(cons_matrix)

In [ ]:
cons_matrix.mean()

In [ ]:
def coph_cor(X, model, n_runs = 1):
        """
        Compute cophenetic correlation coefficient of consensus matrix, generally obtained from multiple NMF runs. 
        
        The cophenetic correlation coefficient is measure which indicates the dispersion of the consensus matrix and is based 
        on the average of connectivity matrices. It measures the stability of the clusters obtained from NMF. 
        It is computed as the Pearson correlation of two distance matrices: the first is the distance between samples induced by the 
        consensus matrix; the second is the distance between samples induced by the linkage used in the reordering of the consensus 
        matrix [Brunet2004]_.
        
        Return real number. In a perfect consensus matrix, cophenetic correlation equals 1. When the entries in consensus matrix are
        scattered between 0 and 1, the cophenetic correlation is < 1. We observe how this coefficient changes as factorization rank 
        increases. We select the first rank, where the magnitude of the cophenetic correlation coefficient begins to fall [Brunet2004]_.
        
        :param idx: Used in the multiple NMF model. In factorizations following standard NMF model or nonsmooth NMF model
                    :param:`idx` is always None.
        :type idx: None or `str` with values 'coef' or 'coef1' (`int` value of 0 or 1, respectively) 
        """
        A = consensus(n_runs, X, model)
        # upper diagonal elements of consensus
        avec = np.array([A[i, j] for i in range(A.shape[0] - 1)
                        for j in range(i + 1, A.shape[1])])
        # consensus entries are similarities, conversion to distances
        Y = 1 - avec
        Z = linkage(Y, method='average')
        # cophenetic correlation coefficient of a hierarchical clustering
        # defined by the linkage matrix Z and matrix Y from which Z was
        # generated
        return (cophenet(Z, Y)[0], cophenet(Z,Y)[1])

In [ ]:
from scipy.cluster.hierarchy import linkage, cophenet

In [ ]:
%%time
c, D = coph_cor(LL_norm, model, n_runs = 10)

In [ ]:
from scipy.spatial.distance import squareform

In [ ]:
plt.imshow(squareform(D))
plt.title('Cophenetic Correlation is {}'.format(round(c,2)))

In [ ]:
# Notes:
# for 1000 iterations I get a warning and returns nans
# most probably division by zero (not clear which run)
# the connectivity is the same for each observation: i.e. all observations are assigned to the same component
# so the variance becomes zero and that results in dividing by zero
# I think I need some normalization
# It works when smoothness and sparsity are lower and there is more variation in the coefficients and they intersect each other.

In [ ]:
# matlab version 
function seq = find_match_factor_seq(rho,rank)
% Find matching component sequence based on correlation coefficients
%
% Inputs
%   rho   correlation coefficients (based on H or W, H seems to give better results)
%   rank  rank of the decomposition
%
% Outputs
%   seq   matching sequence
%
% Wu-Jung Lee | leewujung@gmail.com
% 2019 05 03

icomp = 1;
while icomp<=rank
    if icomp==1
        seq = nan(rank,1);
    end
    [~,max_ind] = max(rho(:));
    [x_ind,y_ind] = ind2sub(size(rho),max_ind);
    seq(x_ind) = y_ind;

    
    icomp = icomp+1;
end

In [ ]:
def find_match_factor_seq(rho, n_components):
    """
    Find matching component sequence based on correlation coefficients

    Inputs
       rho   correlation coefficients (based on H or W, H seems to give better results)
       rank  rank of the decomposition

    Outputs
       seq   matching sequence
    """
    seq = {}
    for c in range(n_components):
        ind = np.argmax(rho)
        row_ind, col_ind = np.unravel_index(ind, rho.shape)
        seq[row_ind] = col_ind
        rho[row_ind,:] = -2
        rho[:,col_ind] = -2
        
    return (seq)
        
    

In [ ]:
rho = np.corrcoef(model.H.T,model.H.T)

In [ ]:
seq = find_match_factor_seq(rho, 3)

In [ ]:
plt.imshow(rho)

In [ ]:
rho.min()

In [ ]:
# normalize X
output['LL']

LL_norm = nan(size(LL));
sigma_all = nan(size(LL,1),1);
for irow = 1:size(LL,1)
    sigma = std(LL(irow,:));
    LL_norm(irow,:) = LL(irow,:)/sigma;
    sigma_all(irow) = sigma;
end
LL = LL_norm;  % use normalized data for decomposition

In [ ]:
# normalize the pixel values over time
LL_norm = (output['LL'].T/np.linalg.norm(output['LL'],axis=1)).T

In [ ]:
# run for different ranks
c_dict = {}
D_dict = {}
for r in range(2,6):
    model.rank = r
    c, D = coph_cor(output['LL'], model, n_runs = 10)
    c_dict[r] = c
    D_dict[r] = D
    
    
    

In [ ]:
plt.bar(*zip(*c_dict.items()))

In [ ]:
c_dict

In [ ]:
plt.imshow(squareform(D_dict[5]))